# Working with Models

There are different ways to build your deep learning model. You can achieve it using:

- Supervised learning
- Unsupervised learning
- Semi-supervised learning. 

Despite the one you chose, you're going to use the same **pipeline** to **train, test, and deploy** your deep learning model. 

The process begins with the data preparation stage. As it's name suggests, we load the generic data, which can be in many different formats, such as text, images, videos, audio files, etc, from an external source, and we convert it to numeric values suitable for model training. These numeric values are in form of tensors. Then tensors need to be pre-processed during transforms, and we group them with batches that can be passed into the model. 

### Data Preparation

Data preparation is the first step in developing a deep learning model. This step consists of loading the data, applying transforms, and batching the data using PyTorch built-in capabilities.

We are going to use an existing popular academic dataset called CIFAR-10, developed by researchers from the Canadian Institute for Advanced Research. CIFAR-10 dataset is a subset of a much larger dataset with 80 million images in it. It consists of 60,000 small color photographs of objects from 10 classes divided into 50,000 training images and 10,000 test images. Here is the table with class labels and their associated integer values.


| Integer Value | Class Label   |
| ------------- | ------------- |
|       0       | airplane      |
|       1       | automobile    |
|       2       | bird          |
|       3       | cat           |
|       4       | deer          |
|       5       | dog           |
|       6       | frog          |
|       7       | horse         |
|       8       | ship          |
|       9       | truck         |

The Torchvision datasets module provides several subclasses to load image data from standard data sets.

### Data Loading


In [ ]:
pip install torchvision

In [ ]:
pip install tensorflow

In [ ]:
pip install keras

In [ ]:
# 
import torch
from torchvision.datasets import CIFAR10
from keras.datasets import cifar10
from matplotlib import pyplot

train_data = CIFAR10(root="./train/", train=True, download=True)

In [ ]:
# Load datasets
(trainX, trainY), (testX, testY) = cifar10.load_data()
# Summarize loaded dataset
print('Train: X=%s, y=%s' % (trainX.shape, trainY.shape))
print('Test: X=%s, y=%s' % (testX.shape, testY.shape))

In [ ]:
# Display images
for i in range(16):
    # define subplot
    pyplot.subplot(4,4,i+1)
    # plot raw pixel data
    pyplot.imshow(trainX[i])
    # show the figure
    pyplot.show()

#### Examine the training dataset

When we print this shape, we see we have 50,000 images divided into 10 classes and corresponding labels. The label is an integer value that represents the class of the image. For example, airplane zero, automobile one, birds two, etc.

In [ ]:
print(train_data.data.shape)
print(train_data.class_to_idx)

### Data Transforms

Before passing the data into the model, usually we need to adjust it. For example, data values need to be converted from one type of object to a tensor. 

Another example of adjustment is data values that may be augmented to create a larger dataset. We can achieve this adjustment by applying transforms. PyTorch has a **torchvision library** that supports common computer vision transformations in the **_torchvision.transforms_** and **_torchvision.transforms.v2_** models. 

Transforms are common image transformations that we can use to transform or augment data for training or inference of different tasks. For example: image classification, detection, segmentation, and video classification. 

In [3]:
from torchvision import transforms
from torchvision.datasets import CIFAR10

train_data_path = "./train/"
train_transforms = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
    # These values are standard for CIFAR-10
    # Without normalization, these values would typically range from zero to 255 
    # for each channel, representing the intensity of RGB component of each pixel
    transforms.Normalize(
        mean= (0.4914, 0.4822, 0.4465),
        std= (0.2023, 0.1994, 0.2010)
    )
])

training_data = CIFAR10(train_data_path,
                        train= True,
                        download= True,
                        transform= train_transforms)

# Take a look at the transforms
# Training data of first image
print(training_data[0])

Files already downloaded and verified
(tensor([[[-1.2854, -1.3629, -1.5180,  ...,  0.4981,  0.4593,  0.4399],
         [-1.4986, -1.5761, -1.7312,  ...,  0.3430,  0.3236,  0.3236],
         [-1.9057, -1.9832, -2.1383,  ...,  0.0522,  0.0522,  0.0716],
         ...,
         [ 1.0408,  1.0021,  0.9439,  ..., -0.3549, -0.5293, -0.6263],
         [ 1.0214,  0.9827,  0.8858,  ...,  0.1297, -0.1223, -0.2386],
         [ 1.0021,  0.9633,  0.8664,  ...,  0.3624,  0.0910, -0.0447]],

        [[-1.1989, -1.2776, -1.4349,  ...,  0.0401,  0.0204,  0.0204],
         [-1.3956, -1.4939, -1.6512,  ..., -0.1566, -0.1566, -0.1566],
         [-1.8086, -1.9069, -2.1036,  ..., -0.5696, -0.5302, -0.5302],
         ...,
         [ 0.3351,  0.2564,  0.1188,  ..., -0.9826, -1.1202, -1.1792],
         [ 0.3941,  0.3154,  0.1778,  ..., -0.4712, -0.6876, -0.8056],
         [ 0.4138,  0.3351,  0.1974,  ..., -0.2156, -0.4712, -0.6089]],

        [[-0.9922, -1.0703, -1.2459,  ..., -0.2313, -0.2118, -0.2118],
      

### Model Development and Training

After preparing data sets, we can finally explore model development. It consists of a few steps:

- Model design
- Training
- Validation
- Testing

#### Import libraries and dataset

In [4]:
import torch
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchvision import models
from torch import optim
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

#### Define neural network, **init** and forward functions

In [15]:
# Define neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    # The forward function defines how data flows through the network 
    # in both training and making predictions
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

#### Instantiate the Model

In [16]:
net = Net()

# define the Loss Function adn Optimizer
loss_function = nn.CrossEntropyLoss()
optimaizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

#### Load and transform the data

In [17]:
import torch.utils
import torch.utils.data


transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        (0.5, 0.5, 0.5),
        (0.5, 0.5, 0.5)
    )
])

train_set = CIFAR10(root='./data',
                   train=True,
                   download=True,
                   transform=transform)

trainloader = torch.utils.data.DataLoader(train_set, batch_size=4, shuffle=True, num_workers=2)

test_set = CIFAR10(root='./data',
                   train=False,
                   download=True,
                   transform=transform)

testloader = torch.utils.data.DataLoader(test_set, batch_size=4, shuffle=False, num_workers=2)



Files already downloaded and verified
Files already downloaded and verified


#### Train the Network

In the training step, our model learns by adjusting its weights based on the computed loss and gradients. It involves iterating over a data set multiple times, which we call epochs.

In [19]:
# Train the network (10 minutes more or less to run the below code)

# loop over the dataset multiple times
for epoch in range(10):
    # Initialize a variable to accumulate the loss over each batch
    running_loss = 0.0
    # This inner loop iterates over the training data set 
    # and the train loader provides batches of data, meaning images and labels
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a tuple of [input, labels]
        inputs, labels = data

        # zero the parameters gradients
        optimaizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        # After we pass the input data through the network to get outputs, 
        # we compute the loss by comparing the model's outputs with the actual labels.
        loss = loss_function(outputs, labels)
        # By calling loss.backward function, we perform a backward pass to compute 
        # the gradient of the loss with respect to the network parameters.
        loss.backward()
        optimaizer.step()

        # print statistics
        running_loss += loss.item()
        # print every 2,000 batches
        if i%2000 == 1999:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')

print('Finished Training')

[1,  2000] loss: 2.174
[1,  4000] loss: 3.973
[1,  6000] loss: 5.624
[1,  8000] loss: 7.207
[1, 10000] loss: 8.703
[1, 12000] loss: 10.157
[2,  2000] loss: 1.384
[2,  4000] loss: 2.737
[2,  6000] loss: 4.073
[2,  8000] loss: 5.371
[2, 10000] loss: 6.646
[2, 12000] loss: 7.896
[3,  2000] loss: 1.188
[3,  4000] loss: 2.358
[3,  6000] loss: 3.523
[3,  8000] loss: 4.679
[3, 10000] loss: 5.848
[3, 12000] loss: 7.003
[4,  2000] loss: 1.072
[4,  4000] loss: 2.150
[4,  6000] loss: 3.229
[4,  8000] loss: 4.319
[4, 10000] loss: 5.390
[4, 12000] loss: 6.451
[5,  2000] loss: 0.990
[5,  4000] loss: 2.015
[5,  6000] loss: 3.006
[5,  8000] loss: 4.014
[5, 10000] loss: 5.045
[5, 12000] loss: 6.057
[6,  2000] loss: 0.918
[6,  4000] loss: 1.880
[6,  6000] loss: 2.826
[6,  8000] loss: 3.812
[6, 10000] loss: 4.780
[6, 12000] loss: 5.747
[7,  2000] loss: 0.859
[7,  4000] loss: 1.761
[7,  6000] loss: 2.674
[7,  8000] loss: 3.614
[7, 10000] loss: 4.537
[7, 12000] loss: 5.476
[8,  2000] loss: 0.855
[8,  4000]

### Validation and Testing

Validation and testing are important parts of model development as they take care that overfitting does not occur and that the model performs well against unseen data.

In the validation step, we use a separate set of data, which we haven't used previously in training and call this set validation set. The main goal is tuning hip parameters such as learning rate or number of epochs, so we can provide an early indication of how our model is performing. 

For this stage, we will need to modify the code on the load and transform section a bit as well as the Train the Network:

In [20]:
# Refactor code for the Load and Transform step
import torch.utils
import torch.utils.data
from torch.utils.data import random_split

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        (0.5, 0.5, 0.5),
        (0.5, 0.5, 0.5)
    )
])

train_transforms = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
      mean=(0.4914, 0.4822, 0.4465),
      std=(0.2023, 0.1994, 0.2010)
    )
])

train_data = CIFAR10(root="./train/",
                     train=True,
                     download=True, 
                     transform=train_transforms)

# trainset = CIFAR10(root='./data',
#                    train=True,
#                    download=True,
#                    transform=transform)

train_set, val_set = random_split(train_data,[40000, 10000])

# trainloader = torch.utils.data.DataLoader(
#     train_set, 
#     batch_size=4, 
#     shuffle=True, 
#     num_workers=2)

trainloader = torch.utils.data.DataLoader(
    train_set,
    batch_size=16,
    shuffle=True)

valloader = torch.utils.data.DataLoader(
    val_set,
    batch_size=16,
    shuffle=True)

# The test set is a separate data set that the model has never seen during training. 
# It provides us with a final evaluation of the model's performance. 
test_set = CIFAR10(root='./data',
                   train=False,
                   download=True,
                   transform=transform)

testloader = torch.utils.data.DataLoader(test_set, batch_size=4, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [21]:
# Refactor code for the Train Network step

for epoch in range(10):
    # Set the model to training mode
    net.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimaizer.zero_grad()
        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimaizer.step()
        running_loss += loss.item()

    net.eval()  # Set the model to evaluation mode for validation
    validation_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            loss = loss_function(outputs, labels)
            validation_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

print(f'Epoch {epoch + 1}, Training Loss: {running_loss / len(trainloader)}, Validation Loss: {validation_loss / len(valloader)}, Validation Accuracy: {100 * correct / total}%')


Epoch 10, Training Loss: 1.0420414512515068, Validation Loss: 8.5669294257164, Validation Accuracy: 35.77%
